In [206]:
import numpy as np
from cma import CMAEvolutionStrategy

def CRP_gen_APUF(num=1200, stages=160, xor_num=11, seed=None):
    np.random.seed(seed)
    chal = np.random.choice([0, 1], [num, stages]).astype(np.int32)
    para_delay = np.random.randn(xor_num, 2, stages)
    resp_tot = np.zeros([xor_num, num, 160], dtype=np.int32)  
    resp = np.zeros((num, 160), dtype=np.int32)

    for apuf_inx in range(xor_num):
        para = para_delay[apuf_inx]

        delay_sum = np.zeros(num)
        for i in range(stages):
            para_select = para[chal[:, i], i]
            delay_sum = delay_sum * (1 - 2 * chal[:, i]) + para_select

        resp_tot[apuf_inx, delay_sum > 0, :resp.shape[1]] = 1
        resp[:, :resp_tot.shape[2]] = np.bitwise_xor(resp[:, :resp_tot.shape[2]], resp_tot[apuf_inx])

    return chal, resp, para_delay, resp_tot

def fitness_fun(weight, chal, resp, xor_num):
    stage_num = chal.shape[1]
    crp_num = chal.shape[0]
    if weight.shape[0] != (xor_num * (stage_num + 1)):
        print('Warning: weight.shape[0] != (xor_num * (stage_num + 1)):')
    delay_para = weight.reshape(stage_num + 1, xor_num)
    C = 1 - 100 * chal
    C_linear = np.ones((crp_num, stage_num + 1))
    for i in range(stage_num - 1, -1, -1):
        C_linear[:, i] = C_linear[:, i + 1] * C[:, i]
    
    delay_sum = np.matmul(C_linear, delay_para)
    sum_prod = np.prod(delay_sum, axis=1)
    
    resp_predict = np.zeros((crp_num, 160), dtype=np.int8)
    resp_predict[sum_prod > 0, :resp.shape[1]] = 1

    err_rate = np.sum(resp ^ resp_predict) / (crp_num * resp.shape[1]) * 200
    return err_rate

train_num = 200
test_num = 2000
stage_num = 160  
xor_num = 10
train_iterations = 100

crp_num = train_num + test_num

res = CRP_gen_APUF(crp_num, stage_num, xor_num, 10)

chal_train = res[0][:train_num]
resp_train = res[1][:train_num]
chal_test = res[0][train_num:]
resp_test = res[1][train_num:]

weight_size = (stage_num + 1) * xor_num
es = CMAEvolutionStrategy(weight_size * [0], 1)
es.optimize(fitness_fun, iterations=train_iterations, verb_disp=10, args=(chal_train, resp_train, xor_num))
res = es.result

error_rate = fitness_fun(res[0], chal_test, resp_test, xor_num)
print('Prediction Accuracy: ', 100 - error_rate, '%')


(13_w,26)-aCMA-ES (mu_w=7.6,w_1=23%) in dimension 1610 (seed=391019, Wed Mar 13 19:52:59 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     26 8.400000000000000e+01 1.0e+00 9.95e-01  1e+00  1e+00 0:00.1
    2     52 8.000000000000000e+01 1.0e+00 9.90e-01  1e+00  1e+00 0:00.2
    3     78 8.200000000000000e+01 1.0e+00 9.85e-01  1e+00  1e+00 0:00.4
   10    260 8.300000000000000e+01 1.0e+00 9.56e-01  1e+00  1e+00 0:01.1
   20    520 8.200000000000000e+01 1.0e+00 9.24e-01  9e-01  9e-01 0:03.2
   30    780 8.500000000000000e+01 1.0e+00 8.97e-01  9e-01  9e-01 0:05.2
   40   1040 8.200000000000000e+01 1.0e+00 8.76e-01  9e-01  9e-01 0:07.3
   50   1300 8.100000000000000e+01 1.0e+00 8.58e-01  9e-01  9e-01 0:09.4
   60   1560 8.100000000000000e+01 1.0e+00 8.42e-01  8e-01  8e-01 0:10.3
   70   1820 8.100000000000000e+01 1.0e+00 8.29e-01  8e-01  8e-01 0:12.4
   80   2080 8.100000000000000e+01 1.0e+00 8.18e-01  8e-01  8e-01 0:14.5
   90   2340 8.100000000000000